# Import Modules 

In [1]:
from model.unet import Unet
from dataset import SegmentationDataset
from dice_score import *
from evaluate import * 
from torch.utils.data import DataLoader 
from torch.utils.data import random_split 
from tqdm import tqdm 
import torch
import torch.nn.functional as F 
import albumentations as A 

# Parameter Set

In [2]:
BATCH_SIZE_TRAIN = 8
BATCH_SIZE_TEST = 8
learning_rate = 0.0003
epochs = 1000

# DATASET Defining

In [3]:
train_transforms = A.Compose(
    [
    A.Rotate(),      
    A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.5),

    A.OneOf([
            A.OpticalDistortion(p=0.3),
            A.GridDistortion(p=.1),
            A.IAAPiecewiseAffine(p=0.3),
            A.ElasticTransform(p=0.3)
        ], p=0.5),
    ]
)
dataset = SegmentationDataset()

In [4]:
dataset_train, dataset_test = random_split(dataset, [950, 50], generator = torch.Generator().manual_seed(0))
dataset_train.dataset.transforms= train_transforms 
train_loader = DataLoader(dataset_train, BATCH_SIZE_TRAIN, shuffle = True)
test_loader = DataLoader(dataset_test, BATCH_SIZE_TEST, shuffle = False)

# Model Creation

In [5]:
model = Unet()

Using cache found in C:\Users\mHealth-11/.cache\torch\hub\milesial_Pytorch-UNet_master


In [6]:
print(model)

Unet(
  (model): UNet(
    (inc): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (down1): Down(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): DoubleConv(
          (double_conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
            (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1

# Training Phase

In [7]:
optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate, weight_decay = 1e-8, momentum = 0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience = 2)
criterion = torch.nn.BCELoss()

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
model.n_classes = 1
model = model.to(device)

In [10]:
global_step = 0
compare_val_score = 0 
for epoch in range(epochs):
    model.train()
    epoch_loss = 0 
    pbar = tqdm(train_loader)
    for batch, (images, true_masks) in enumerate(pbar):
        images = images.to(device).float()
        true_masks = true_masks.to(device).float()
        pred_masks = model(images)
        
        loss1 = criterion(torch.sigmoid(pred_masks), true_masks)
        loss2 = dice_loss(torch.sigmoid(pred_masks).float(),
                         true_masks.float())
        loss = loss1 + loss2 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        global_step +=1 
        
        pbar.set_postfix({'loss': loss.item(),'epoch':epoch,'step':global_step,'c':compare_val_score})
        division_step = (950 // (10 * BATCH_SIZE_TRAIN))
        if division_step >0:
            if global_step % division_step ==0:
                val_score = evaluate(model,test_loader, device, train_transforms)
                scheduler.step(val_score)
                print(val_score.item())
                if val_score.item() > compare_val_score:
                    model.val_score = val_score.item()
                    torch.save(model, 'best.ckpt')
                    compare_val_score = val_score.item()
                
        
        

  8%|███▊                                          | 10/119 [00:07<01:11,  1.53it/s, loss=0.781, epoch=0, step=11, c=0]c:\users\mhealth-11\appdata\local\programs\python\python36\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  9%|████▎                                         | 11/119 [00:10<02:40,  1.49s/it, loss=0.781, epoch=0, step=11, c=0]

0.5120354890823364


 18%|███████▍                                  | 21/119 [00:17<01:07,  1.46it/s, loss=0.775, epoch=0, step=22, c=0.512]

0.5933647155761719


 28%|███████████▋                              | 33/119 [00:31<02:03,  1.44s/it, loss=0.504, epoch=0, step=33, c=0.593]

...

0.8649364709854126


 72%|████████████████████████████▏          | 86/119 [01:18<00:48,  1.48s/it, loss=0.463, epoch=21, step=2585, c=0.878]

0.8465710282325745


 81%|███████████████████████████████▍       | 96/119 [01:28<00:21,  1.09it/s, loss=0.334, epoch=21, step=2596, c=0.878]


KeyboardInterrupt: 

In [ ]:
# bce + dice loss 

In [ ]:
# dice score  as matrix 